# Bandpass Sampling

This notebook demonstrates the bandpass sampling capability in PySM, which allows resampling bandpasses using kernel density estimation (KDE). This technique is useful for generating realistic variations in detector bandpasses for simulation studies.

## Theory

The bandpass sampling process involves:

1. **Normalization**: Normalize bandpass so $\int b(\nu) d\nu = 1$
2. **CDF Creation**: Compute cumulative distribution function
3. **Bootstrap Resampling**: Draw samples from uniform distribution and map to frequencies
4. **Kernel Density Estimation**: Apply Gaussian KDE with optimized bandwidth
5. **Moment Calculation**: Compute centroid and bandwidth for each resampled bandpass

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pysm3

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Create a Synthetic Bandpass

Create a realistic bandpass with primary and secondary lobes.

In [ ]:
# Create frequency grid
nu = np.linspace(80, 120, 200)

# Primary Gaussian at 100 GHz
primary = np.exp(-0.5 * ((nu - 100) / 8)**2)
secondary = 0.1 * np.exp(-0.5 * ((nu - 110) / 3)**2)

# Add noise
np.random.seed(42)
noise = 0.02 * np.random.randn(len(nu))
bnu = np.maximum(primary + secondary + noise, 0)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(nu, bnu, 'k-', linewidth=2, label='Original')
plt.xlabel('Frequency [GHz]')
plt.ylabel('Transmission')
plt.title('Synthetic Input Bandpass')
plt.legend()
plt.grid(True, alpha=0.3)

## Compute Original Bandpass Moments

In [ ]:
try:
    from numpy import trapezoid
except ImportError:
    from numpy import trapz as trapezoid

bnu_norm = bnu / trapezoid(bnu, nu)
orig_centroid, orig_bandwidth = pysm3.compute_moments(nu, bnu_norm)

print(f"Original bandpass:")
print(f"  Centroid:  {orig_centroid:.4f} GHz")
print(f"  Bandwidth: {orig_bandwidth:.4f} GHz")

## Resample the Bandpass

Generate 6 resampled bandpasses for different wafers.

In [ ]:
num_wafers = 6
results = pysm3.resample_bandpass(
    nu, bnu,
    num_wafers=num_wafers,
    bootstrap_size=128,
    random_seed=1929
)

print(f"Resampled {num_wafers} bandpasses\n")
print(f"{'Wafer':<8} {'Centroid':<18} {'Bandwidth':<18}")
print("-" * 44)
for i, r in enumerate(results):
    print(f"{i:<8} {r['centroid']:>16.4f}  {r['bandwidth']:>16.4f}")

## Visualize Resampled Bandpasses

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (ax, result) in enumerate(zip(axes, results)):
    ax.plot(nu, bnu_norm, 'k--', linewidth=1.5, alpha=0.5, label='Original')
    ax.plot(result['frequency'], result['weights'], 
            'C1-', linewidth=2, alpha=0.8, label=f"Wafer {i}")
    ax.set_xlabel('Frequency [GHz]')
    ax.set_ylabel('Transmission')
    ax.set_title(f"Wafer {i}: {result['centroid']:.2f} GHz")
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()

## Statistical Distribution

Generate many resampled bandpasses to examine moment distributions.

In [ ]:
n_samples = 50
results_many = pysm3.resample_bandpass(
    nu, bnu, num_wafers=n_samples, bootstrap_size=128, random_seed=1929
)

centroids = [r['centroid'] for r in results_many]
bandwidths = [r['bandwidth'] for r in results_many]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.hist(centroids, bins=20, alpha=0.7, color='C0', edgecolor='black')
ax1.axvline(orig_centroid, color='red', linestyle='--', linewidth=2, 
            label=f'Original: {orig_centroid:.2f}')
ax1.set_xlabel('Centroid [GHz]')
ax1.set_title(f'Centroid Distribution (N={n_samples})')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.hist(bandwidths, bins=20, alpha=0.7, color='C1', edgecolor='black')
ax2.axvline(orig_bandwidth, color='red', linestyle='--', linewidth=2,
            label=f'Original: {orig_bandwidth:.2f}')
ax2.set_xlabel('Bandwidth [GHz]')
ax2.set_title(f'Bandwidth Distribution (N={n_samples})')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
print(f"Centroid:  mean={np.mean(centroids):.4f}, std={np.std(centroids):.4f}")
print(f"Bandwidth: mean={np.mean(bandwidths):.4f}, std={np.std(bandwidths):.4f}")